## Libraries

In [1]:
import pulp as lp
import random

## Src

In [2]:
def generate_vrptw_parameters(
    num_vehicles,
    num_customers,
    cost_range=(10, 50),
    time_range=(5, 30),
    demand_range=(5, 15),
    capacity=20,
    time_window_range=(0, 100),
    large_constant=1000,
    seed=None,
):
    """
    Generates random VRPTW parameters for a given number of vehicles and customers.

    Parameters:
    - num_vehicles (int): Number of vehicles.
    - num_customers (int): Number of customers (not including depots).
    - cost_range (tuple): Range for travel costs between nodes.
    - time_range (tuple): Range for travel times between nodes.
    - demand_range (tuple): Range for customer demands.
    - capacity (int): Capacity of each vehicle.
    - time_window_range (tuple): Range for time windows (start and end times).
    - large_constant (int): Large constant for constraints.

    Returns:
    - dict: A dictionary containing all sets and parameters.
    """
    if seed is not None:
        random.seed(seed)
    # Sets
    V = list(range(1, num_vehicles + 1))  # Vehicles labeled from 1 to num_vehicles
    N = list(
        range(num_customers + 2)
    )  # Nodes labeled from 0 to num_customers+1 (including depots)
    C = list(range(1, num_customers + 1))  # Customers labeled from 1 to num_customers

    # Parameters
    cij = {(i, j): random.randint(*cost_range) for i in N for j in N if i != j}
    tij = {(i, j): random.randint(*time_range) for i in N for j in N if i != j}
    di = {i: random.randint(*demand_range) for i in C}  # Customer demands
    q = capacity  # Vehicle capacity
    a = {i: random.randint(*time_window_range) for i in N}  # Start of time window
    b = {
        i: random.randint(a[i] + 1, time_window_range[1]) for i in N
    }  # End of time window, after start time
    K = large_constant  # Large constant for time window constraints

    # Ensuring depot time windows
    a[0], b[0] = 0, time_window_range[1]  # Starting depot time window
    a[num_customers + 1], b[num_customers + 1] = (
        0,
        time_window_range[1],
    )  # Returning depot time window

    return {
        "V": V,
        "N": N,
        "C": C,
        "cij": cij,
        "tij": tij,
        "di": di,
        "q": q,
        "a": a,
        "b": b,
        "K": K,
    }

## Constants

In [3]:
n = 3  # num of customers
num_vehicles = 2
parameters = generate_vrptw_parameters(
    num_vehicles=num_vehicles, num_customers=n, seed=30
)
print(parameters)
V, N, C, cij, tij, di, q, a, b, K = parameters.values()

{'V': [1, 2], 'N': [0, 1, 2, 3, 4], 'C': [1, 2, 3], 'cij': {(0, 1): 44, (0, 2): 28, (0, 3): 49, (0, 4): 11, (1, 0): 49, (1, 2): 23, (1, 3): 26, (1, 4): 13, (2, 0): 35, (2, 1): 34, (2, 3): 18, (2, 4): 15, (3, 0): 39, (3, 1): 10, (3, 2): 43, (3, 4): 25, (4, 0): 11, (4, 1): 14, (4, 2): 20, (4, 3): 48}, 'tij': {(0, 1): 21, (0, 2): 17, (0, 3): 25, (0, 4): 16, (1, 0): 22, (1, 2): 7, (1, 3): 17, (1, 4): 5, (2, 0): 12, (2, 1): 30, (2, 3): 23, (2, 4): 26, (3, 0): 13, (3, 1): 29, (3, 2): 18, (3, 4): 23, (4, 0): 8, (4, 1): 26, (4, 2): 13, (4, 3): 25}, 'di': {1: 13, 2: 6, 3: 14}, 'q': 20, 'a': {0: 0, 1: 39, 2: 16, 3: 39, 4: 0}, 'b': {0: 100, 1: 72, 2: 27, 3: 48, 4: 100}, 'K': 1000}


In [4]:
# for i in N:
#     a[i], b[i] = 0, 100

In [5]:
print(a)
print(b)

{0: 0, 1: 39, 2: 16, 3: 39, 4: 0}
{0: 100, 1: 72, 2: 27, 3: 48, 4: 100}


Modèle de minimisation de coût

In [6]:
problem = lp.LpProblem("VRPTW", lp.LpMinimize)

Dicision variables

In [7]:
x = lp.LpVariable.dicts("x", ((k, i, j) for k in V for i in N for j in N if i != j), cat='Binary')
s = lp.LpVariable.dicts("s", ((i, k) for i in N for k in V), lowBound=0, cat='Continuous')

Objective function

In [8]:
problem += lp.lpSum(cij[i, j] * x[k, i, j] for k in V for i in N for j in N if i != j)

## Constraints

Each customer is visited exactly once

In [9]:
for i in C:
    problem += lp.lpSum(x[k, i, j] for k in V for j in N if i != j) == 1

In [10]:
problem += lp.lpSum(x[k, 0, 1] for k in V) == 0

Vehicle capacity constraint

In [11]:

for k in V:
    problem += lp.lpSum(di[i] * x[k, i, j] for i in C for j in N if i != j) <= q


Each vehicle starts at the depot (node 0)

In [12]:
for k in V:
    problem += lp.lpSum(x[k, 0, j] for j in N if j != 0) == 1

Flow conservation for each customer

In [13]:
for k in V:
    for h in C:
        problem += lp.lpSum(x[k, i, h] for i in N if i != h) == lp.lpSum(
            x[k, h, j] for j in N if h != j
        )


Each vehicle ends at the returning depot (node n+1)

In [14]:
for k in V:
    problem += lp.lpSum(x[k, i, n + 1] for i in N if i != n + 1) == 1

In [15]:
problem

VRPTW:
MINIMIZE
44*x_(1,_0,_1) + 28*x_(1,_0,_2) + 49*x_(1,_0,_3) + 11*x_(1,_0,_4) + 49*x_(1,_1,_0) + 23*x_(1,_1,_2) + 26*x_(1,_1,_3) + 13*x_(1,_1,_4) + 35*x_(1,_2,_0) + 34*x_(1,_2,_1) + 18*x_(1,_2,_3) + 15*x_(1,_2,_4) + 39*x_(1,_3,_0) + 10*x_(1,_3,_1) + 43*x_(1,_3,_2) + 25*x_(1,_3,_4) + 11*x_(1,_4,_0) + 14*x_(1,_4,_1) + 20*x_(1,_4,_2) + 48*x_(1,_4,_3) + 44*x_(2,_0,_1) + 28*x_(2,_0,_2) + 49*x_(2,_0,_3) + 11*x_(2,_0,_4) + 49*x_(2,_1,_0) + 23*x_(2,_1,_2) + 26*x_(2,_1,_3) + 13*x_(2,_1,_4) + 35*x_(2,_2,_0) + 34*x_(2,_2,_1) + 18*x_(2,_2,_3) + 15*x_(2,_2,_4) + 39*x_(2,_3,_0) + 10*x_(2,_3,_1) + 43*x_(2,_3,_2) + 25*x_(2,_3,_4) + 11*x_(2,_4,_0) + 14*x_(2,_4,_1) + 20*x_(2,_4,_2) + 48*x_(2,_4,_3) + 0
SUBJECT TO
_C1: x_(1,_1,_0) + x_(1,_1,_2) + x_(1,_1,_3) + x_(1,_1,_4) + x_(2,_1,_0)
 + x_(2,_1,_2) + x_(2,_1,_3) + x_(2,_1,_4) = 1

_C2: x_(1,_2,_0) + x_(1,_2,_1) + x_(1,_2,_3) + x_(1,_2,_4) + x_(2,_2,_0)
 + x_(2,_2,_1) + x_(2,_2,_3) + x_(2,_2,_4) = 1

_C3: x_(1,_3,_0) + x_(1,_3,_1) + x_(1,_3,_2) + x_

Time window constraint with large constant K

In [16]:
for k in V:
    for i in N:
        for j in N:
            if i != j:
                problem += s[i, k] + tij[i, j] - K * (1 - x[k, i, j]) <= s[j, k]

Start time must be within the time window

In [17]:
for i in N:
    for k in V:
        problem += a[i] <= s[i, k]
        problem += s[i, k] <= b[i]


Binary constraint for x variables

In [18]:
for k in V:
    for i in N:
        for j in N:
            if i != j:
                problem += x[k, i, j] >= 0
                problem += x[k, i, j] <= 1

## Solution

In [19]:
problem.solve()
for var in problem.variables():
    print(f"{var.name} = {var.varValue}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/cuphead/Projects/VRPTW-Meta-Heuristique/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/931426cc06674e92bdc277eb4bc7774d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/931426cc06674e92bdc277eb4bc7774d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 161 COLUMNS
At line 616 RHS
At line 773 BOUNDS
At line 814 ENDATA
Problem MODEL has 156 rows, 50 columns and 334 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 73 - 0.00 seconds
Cgl0002I 2 variables fixed
Cgl0003I 20 fixed, 0 tightened bounds, 32 strengthened rows, 11 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 56 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 sub

Results

In [20]:
print("Solution Status:", lp.LpStatus[problem.status])
print("Objective value:", lp.value(problem.objective))

Solution Status: Optimal
Objective value: 149.0


Get the routes for each vehicle

In [21]:
# sol_var = {}
# for k in V:
#     for i in N:
#         for j in N:
#             if i!=j:
#                 sol_var[(k, i, j)] = lp.value(x[k, i, j])

In [22]:
routes = {k: [] for k in V} 

for k in V:
    current_node = 0  # Start from the depot (node 0)
    while (
        current_node != n + 1
    ):  # Continue until reaching the returning depot (node n+1)
        for j in N:
            #if current_node != j and sol_var[k, current_node, j] == 1:
            if current_node != j and lp.value(x[k, current_node, j]) == 1:
                print(f"j:{j}, k:{k}, current node:{current_node}")
                routes[k].append((current_node, j))
                current_node = j
                break

j:3, k:1, current node:0
j:4, k:1, current node:3
j:2, k:2, current node:0
j:1, k:2, current node:2
j:4, k:2, current node:1


Print the routes

In [23]:
for k in V:
    route_str = " -> ".join(str(node) for node, _ in routes[k]) + f" -> {n + 1}"
    print(f"Route for vehicle {k}: {route_str}")

Route for vehicle 1: 0 -> 3 -> 4
Route for vehicle 2: 0 -> 2 -> 1 -> 4


**Note:** is 0 (n+1 and 0 are bothe the depot)

Display service start times for each customer

In [24]:
for k in V:
    print(f"\nService start times for vehicle {k}:")
    for i in N:
        start_time = lp.value(s[i, k])
        if (
            start_time is not None and start_time > 0
        ):  # Only print if the vehicle visits the customer
            print(f"  Customer {i}: Start time = {start_time:.2f}")


Service start times for vehicle 1:
  Customer 1: Start time = 39.00
  Customer 2: Start time = 16.00
  Customer 3: Start time = 39.00
  Customer 4: Start time = 100.00

Service start times for vehicle 2:
  Customer 1: Start time = 72.00
  Customer 2: Start time = 17.00
  Customer 3: Start time = 39.00
  Customer 4: Start time = 100.00
